In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 1809, done.
remote: Counting objects: 100% (658/658), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 1809 (delta 520), reused 569 (delta 454), pack-reused 1151
Receiving objects: 100% (1809/1809), 158.38 MiB | 22.29 MiB/s, done.
Resolving deltas: 100% (1350/1350), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 22.0 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationImpressions_Cython_Epoch.c: In functio

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv
/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv
/kaggle/input/training-tuned-models/slim_bpr_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/__results__.html
/kaggle/input/training-tuned-models/slim_bpr_all.zip
/kaggle/input/training-tuned-models/__notebook__.ipynb
/kaggle/input/training-tuned-models/ials_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/ials_train_validation.zip
/kaggle/input/training-tuned-models/__output__.json
/kaggle/input/training-tuned-models/slim_bpr_train_validation.zip
/kaggle/input/training-tuned-models/ials_all.zip
/kaggle/input/training-tuned-models/custom.css
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/run_all_algorithms.py
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/cython_example_FunkSVD.py
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/LICENSE
/kaggle/input/training-tuned-models/R

In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender, SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender, PureSVDItemRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from Recommenders.BaseRecommender import BaseRecommender

from Recommenders.Hybrid.LinearCombinationRecommender import LinearCombinationRecommender
from Data_manager.CrossValidationSplitter import CrossValidationSplitter

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

# Configurations

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters': True,
    'database_path': '/kaggle/working/tuning_hybrid.db',
    'copy_prev_tuning_db': True,
    'copy_prev_best_params': True,
    'save_github': True
}

# Tuning

In [10]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [11]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [12]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)

In [13]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_item.json', 'r') as params_file:
    params_item_knn = json.load(params_file)
params_item_knn

{'topK': 14,
 'shrink': 25,
 'similarity': 'tversky',
 'tversky_alpha': 0.18163095712123836,
 'tversky_beta': 1.8529812241698465}

In [14]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_RP3.json', 'r') as params_file:
    params_rp3 = json.load(params_file)
params_rp3

{'alpha': 0.26348679053501006, 'beta': 0.17536156577039744, 'topK': 27}

In [15]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_alpha.json', 'r') as params_file:
    params_p3alpha = json.load(params_file)
params_p3alpha

{'normalize_similarity': 'True',
 'implicit': 'True',
 'alpha': 0.29354071531607107,
 'topK': 49}

In [16]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_hybrid_merging.json', 'r') as params_file:
    params_merging = json.load(params_file)
params_merging

{'alpha': 0.3375591188839621, 'topK': 86}

In [17]:
shutil.copyfile(
    '/kaggle/input/training-tuned-models/ials_train_validation.zip', 
    '/kaggle/working/ials_train_validation.zip', 
)
shutil.copyfile(
    '/kaggle/input/training-tuned-models/ials_train_validation.zip', 
    '/kaggle/working/ials_all.zip', 
)

'/kaggle/working/ials_all.zip'

In [18]:
recommender_item = ItemKNNCFRecommender(URM_train)
recommender_item.fit(**params_item_knn)

recommender_rp3 = RP3betaRecommender(URM_train)
recommender_rp3.fit(**params_rp3)

recommender_p3a = P3alphaRecommender(URM_train)
recommender_p3a.fit(**params_p3alpha)

recommender_slim = SLIMElasticNetRecommender(URM_train)
recommender_slim.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_elastic_train_validation.zip')

recommender_ials = IALSRecommender(URM_train)
recommender_ials.load_model('/kaggle/working/', 'ials_train_validation.zip')

hybrid_similarity = ItemKNNSimilarityHybridRecommender(URM_train, recommender_item.W_sparse, recommender_rp3.W_sparse)
hybrid_similarity.fit(**params_merging)

ItemKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 4580.47 column/sec. Elapsed time 4.88 sec
RP3betaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
RP3betaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2581.41 column/sec. Elapsed time 8.66 sec
P3alphaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
P3alphaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 2989.54 column/sec. Elapsed time 7.48 sec
SLIMElasticNetRecommender: URM Detected 627 ( 4.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 247 ( 1.1%) items with no interactions.
SLIMElasticNetRecommender: Loading model from file '/kaggle/working/Recsys_Challenge_2023/models/slim_elastic_train_validation.zip

In [19]:
class PipelineRecommender(BaseRecommender):
    def __init__(self, URM_train, recommenders, cutoffs):
        super(PipelineRecommender, self).__init__(URM_train)
        self.recommenders = recommenders
        self.cutoffs = cutoffs
        
    def recommend(self, user_ids, cutoff=10, remove_seen_flag=True, remove_top_pop_flag=False, return_scores=True, remove_custom_items_flag=True):
        
        recommendations = []
        scores = np.empty(shape=(0, self.URM_train.shape[1]))
        item_ids = np.arange(self.URM_train.shape[1])
        
        for user_id in user_ids:
            current_recommendations = item_ids.copy()
            for recommender, cut in zip(self.recommenders, self.cutoffs):
                current_recommendations = recommender.recommend(
                    user_id,
                    cutoff=cut,
                    items_to_compute=current_recommendations,
                    remove_seen_flag=remove_seen_flag, 
                    remove_top_pop_flag=remove_top_pop_flag, 
                    return_scores=False, 
                    remove_custom_items_flag=remove_custom_items_flag
                )
                
            recommendations.append(current_recommendations[:cutoff])
            items_score = self.recommenders[-1]._compute_item_score([user_id], current_recommendations[:cutoff])
            scores = np.concatenate([scores, items_score])
            
        if return_scores:
            return np.array(recommendations), scores
        return np.array(recommendations)

In [20]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

print('Merging Models')
result_df, _ = evaluator.evaluateRecommender(hybrid_similarity)
display(result_df)
print('Item KNN')
result_df, _ = evaluator.evaluateRecommender(recommender_item)
display(result_df)
print('RP3 beta')
result_df, _ = evaluator.evaluateRecommender(recommender_rp3)
display(result_df)
print('P3 alpha')
result_df, _ = evaluator.evaluateRecommender(recommender_p3a)
display(result_df)
print('SLIM Elastic')
result_df, _ = evaluator.evaluateRecommender(recommender_slim)
display(result_df)
print('IALS')
result_df, _ = evaluator.evaluateRecommender(recommender_ials)
display(result_df)

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
Merging Models
EvaluatorHoldout: Processed 10456 (100.0%) in 9.11 sec. Users per second: 1148


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.097102,0.165756,0.139351,0.050173,0.085482,0.27428,0.152762,0.114452,0.520658,0.365828,...,0.802826,0.417998,0.802826,0.087463,10.729146,0.998493,0.252365,0.826532,1.477653,0.310478


Item KNN
EvaluatorHoldout: Processed 10456 (100.0%) in 8.89 sec. Users per second: 1176


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.09451,0.161146,0.135378,0.048719,0.082976,0.267977,0.148683,0.111312,0.511572,0.356662,...,0.802826,0.410703,0.802826,0.066309,10.283111,0.997789,0.191329,0.792171,1.734564,0.30426


RP3 beta
EvaluatorHoldout: Processed 10456 (100.0%) in 9.08 sec. Users per second: 1152


...

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.092358,0.15618,0.130929,0.047745,0.081353,0.267656,0.146022,0.108312,0.507269,0.352674,...,0.802826,0.407248,0.802826,0.069984,10.530815,0.998437,0.201933,0.811253,1.356167,0.311674


SLIM Elastic
EvaluatorHoldout: Processed 10456 (100.0%) in 12.29 sec. Users per second: 851


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.097379,0.16502,0.138508,0.051089,0.086077,0.282425,0.154158,0.114358,0.524962,0.375002,...,0.802826,0.421453,0.802826,0.065098,10.631702,0.9989,0.187832,0.819025,1.417992,0.303398


IALS
EvaluatorHoldout: Processed 10456 (100.0%) in 14.73 sec. Users per second: 710


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.08224,0.144123,0.122645,0.041501,0.071584,0.246255,0.131701,0.098458,0.479055,0.316769,...,0.802826,0.384598,0.802826,0.029074,9.74689,0.998532,0.08389,0.750862,1.599865,0.289534


In [21]:
class PipelineRecommender(BaseRecommender):
    def __init__(self, URM_train, recommenders: list[BaseRecommender], cutoffs):
        super(PipelineRecommender, self).__init__(URM_train)
        self.recommenders = recommenders
        self.cutoffs = cutoffs
        
    def recommend(self, user_ids, cutoff=10, remove_seen_flag=True, remove_top_pop_flag=False, return_scores=True, remove_custom_items_flag=True):
        
        recommendations = None
        for recommender_cutoff, recommender in zip(self.cutoffs, self.recommenders):
            recommender_scores = recommender._compute_item_score(user_ids)
            
            for i, user_id in enumerate(user_ids):
                recommender_scores[i, :] = self._remove_seen_on_scores(user_id, recommender_scores[i, :])
            
            if recommendations is None:
                recommendations = np.argsort(recommender_scores, axis=1)[:, -recommender_cutoff:]
                
            else:
                filtered_scores = np.full_like(recommender_scores, -np.inf)
                for i in range(recommendations.shape[0]):
                    row_indices = recommendations[i]
                    filtered_scores[i, row_indices] = recommender_scores[i, row_indices]
                recommendations = np.argsort(filtered_scores, axis=1)[:, -recommender_cutoff:]              
                
        if return_scores:
            return np.flip(recommendations[:, -cutoff:], axis=1), filtered_scores
        return np.flip(recommendations[:, -cutoff:], axis=1)

In [22]:
pipeline_recommender = PipelineRecommender(
    URM_train, 
    recommenders=[recommender_p3a, hybrid_similarity, recommender_slim], 
    cutoffs=[500, 100, 10]
)

result_df, _ = evaluator.evaluateRecommender(pipeline_recommender)
display(result_df)

Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 26.85 sec. Users per second: 389


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.09649,0.161026,0.1345,0.050916,0.085201,0.281619,0.152405,0.112368,0.519606,0.373746,...,0.802826,0.417153,0.802826,0.082387,10.928675,0.999092,0.23772,0.841903,1.190998,0.313387


In [23]:
def objective(trial):
    cutoff_p3alpha = trial.suggest_int('cutoff_p3alpha', 11, 2000)
    cutoff_merging = trial.suggest_int('cutoff_merging', 10, cutoff_p3alpha)
    
    pipeline_recommender = PipelineRecommender(
        URM_train, 
        recommenders=[recommender_p3a, hybrid_similarity, recommender_slim], 
        cutoffs=[cutoff_p3alpha, cutoff_merging, 10]
    )

    result_df, _ = evaluator.evaluateRecommender(pipeline_recommender)
    return result_df.loc[10, 'MAP']

In [24]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_hybrid.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

In [25]:
if config['tune_parameters']:
    study = optuna.create_study(direction='maximize', study_name='hyperparameters_tuning_3stage_pipeline_v2', 
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective, n_trials=100)

[I 2023-12-03 20:05:13,953] Using an existing study with name 'hyperparameters_tuning_3stage_pipeline_v2' instead of creating a new one.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 26.72 sec. Users per second: 391


[I 2023-12-03 20:05:41,009] Trial 300 finished with value: 0.051241031958563193 and parameters: {'cutoff_p3alpha': 46, 'cutoff_merging': 14}. Best is trial 111 with value: 0.051282065587010754.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 26.59 sec. Users per second: 393


[I 2023-12-03 20:06:07,794] Trial 301 finished with value: 0.050132740675970115 and parameters: {'cutoff_p3alpha': 11, 'cutoff_merging': 11}. Best is trial 111 with value: 0.051282065587010754.


...

[I 2023-12-03 20:49:18,040] Trial 397 finished with value: 0.05109966860373381 and parameters: {'cutoff_p3alpha': 48, 'cutoff_merging': 13}. Best is trial 111 with value: 0.051282065587010754.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 26.62 sec. Users per second: 393


[I 2023-12-03 20:49:44,844] Trial 398 finished with value: 0.050969891973622526 and parameters: {'cutoff_p3alpha': 112, 'cutoff_merging': 10}. Best is trial 111 with value: 0.051282065587010754.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 26.46 sec. Users per second: 395


[I 2023-12-03 20:50:11,496] Trial 399 finished with value: 0.050132740675970115 and parameters: {'cutoff_p3alpha': 11, 'cutoff_merging': 11}. Best is trial 111 with value: 0.051282065587010754.


In [26]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [27]:
# if config['tune_parameters']:
#     fig = optuna.visualization.plot_param_importances(study)
#     fig.show()

In [28]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [29]:
if config['tune_parameters']:
    with open('/kaggle/working/best_params_3stage_pipeline.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']: 
        upload_file(
            '/kaggle/working/best_params_3stage_pipeline.json', 
            'tuning_results/best_params_3stage_pipeline.json', 
            'Hybrid Pipeline tuning results (from kaggle notebook)'
        )
    
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/input/Recsys_Challenge_2023/tuning_results/best_params_3stage_pipeline.json', 
        '/kaggle/working/best_params_3stage_pipeline.json'
    )

In [30]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        'tuning_results/tuning_hybrid.db', 
        'Pipeline hybrid recommender tuning db updated results (from kaggle notebook)'
    )

# Training best model

In [31]:
recommender_item = ItemKNNCFRecommender(URM_all)
recommender_item.fit(**params_item_knn)

recommender_rp3 = RP3betaRecommender(URM_all)
recommender_rp3.fit(**params_rp3)

recommender_p3a = P3alphaRecommender(URM_all)
recommender_p3a.fit(**params_p3alpha)

recommender_slim = SLIMElasticNetRecommender(URM_all)
recommender_slim.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_elastic_all.zip')

recommender_ials = IALSRecommender(URM_all)
recommender_ials.load_model('/kaggle/working/', 'ials_all.zip')

hybrid_similarity = ItemKNNSimilarityHybridRecommender(URM_all, recommender_item.W_sparse, recommender_rp3.W_sparse)
hybrid_similarity.fit(**params_merging)

ItemKNNCFRecommender: URM Detected 386 ( 3.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 125 ( 0.6%) items with no interactions.
Similarity column 22347 (100.0%), 4203.27 column/sec. Elapsed time 5.32 sec
RP3betaRecommender: URM Detected 386 ( 3.0%) users with no interactions.
RP3betaRecommender: URM Detected 125 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2603.73 column/sec. Elapsed time 8.58 sec
P3alphaRecommender: URM Detected 386 ( 3.0%) users with no interactions.
P3alphaRecommender: URM Detected 125 ( 0.6%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 2694.99 column/sec. Elapsed time 8.29 sec
SLIMElasticNetRecommender: URM Detected 386 ( 3.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 125 ( 0.6%) items with no interactions.
SLIMElasticNetRecommender: Loading model from file '/kaggle/working/Recsys_Challenge_2023/models/slim_elastic_all.zip'
SLIMElastic

In [32]:
with open('/kaggle/working/best_params_3stage_pipeline.json', 'r') as params_file:
    params = json.load(params_file)
    
pipeline_recommender = PipelineRecommender(
    URM_all, 
    recommenders=[recommender_p3a, hybrid_similarity, recommender_slim],
    cutoffs=[params['cutoff_p3alpha'], params['cutoff_merging'], 10]
)

Recommender_Base_Class: URM Detected 386 ( 3.0%) users with no interactions.
Recommender_Base_Class: URM Detected 125 ( 0.6%) items with no interactions.


In [33]:
recommendations = pipeline_recommender.recommend(user_ids_test['user_id'].unique() - 1, cutoff=10, return_scores=False)
recommendations = [' '.join([str(item + 1) for item in r]).strip() for r in recommendations]

submission_df = pd.DataFrame({'user_id': user_ids_test['user_id'].unique(), 'item_list': recommendations})
display(submission_df.head())
submission_df.to_csv('/kaggle/working/submission.csv', index=False)

,user_id,item_list
0,1,101 36 123 506 403 515 694 1546 254 592
1,2,1095 12 47 1522 50 656 949 102 196 359
2,3,59 2172 857 1281 2748 259 536 584 970 239
3,4,249 28 50 314 171 139 146 254 7 433
4,5,1570 170 95 471 77 131 5138 116 766 7033
